# Importing and displaying VTK data

### Import required libraries:

In [ ]:
import os
import vtk
import numpy as np
from vtk.util import numpy_support
import k3d
from k3d.helpers import download
from ipywidgets import interact, interact_manual, interactive, HBox, VBox
from ipywidgets import widgets
import matplotlib.pyplot as pl

Load vtk as vtp file (see Paraview manual for description of this file format):

### Initialize user interface:

In [45]:
#cms = [attr for attr in dir(k3d.colormaps.matplotlib_color_maps) if not attr.startswith('__')]
cms = ['RdBu','PuOr','Blues','RdYlGn','RdGy','YlOrBr']
vtpdir = 'https://raw.githubusercontent.com/ghalib-bello/myK3D-jupyter/master/examples/'
vtp_dict_urls = {'WallThickness':'exampleLV_wallthick.vtp', 
                 'SystolicExcursion':'exampleLV_distprof.vtp',
                 'Strain': 'exampleLV_Property3.vtp'}
vtp_dict = dict()
for ky in list(vtp_dict_urls.keys()):
    filename = download(vtpdir+vtp_dict_urls[ky])
    LVreader = vtk.vtkXMLPolyDataReader()
    LVreader.SetFileName(filename)
    LVreader.Update()
    vtp_dict[ky]=LVreader.GetOutput()
vtp_dict_keys = list(vtp_dict.keys())

RVreader = vtk.vtkXMLPolyDataReader()
RVreader.SetFileName(download(vtpdir+'exampleRV_endo.vtp'))
RVreader.Update()
RVvtk = RVreader.GetOutput()
RVmesh = k3d.vtk_poly_data(RVvtk, wireframe=True, color=0xD3D3D3, visible=False)

plot2 = k3d.plot()
plot2.wireframe=True
#plot2 += RVmesh
plot2.display()

maps =widgets.Dropdown(options=vtp_dict_keys, value=vtp_dict_keys[0], description='Mesh map to display:')
cmsel=widgets.Select(options=cms,value=cms[0],description='ColorMap', rows=len(cms))
disprv=widgets.Checkbox(value=True,description='Display RV?')

mapsel = '_null_'

@interact(x=disprv)
def f(x):
    global plot2
    if len(plot2.objects)==2:
        plot2.objects[1].visible=x
#    if x & len(plot2.objects)==1:
#        plot2 += RVmesh
#    elif not x & len(plot2.objects)==2:
#        plot2 -= RVmesh
#    else: pass

@interact(x=maps, y=cmsel)
def plot(x, y):
    sh2 = vtp_dict[x]
    scvtk = sh2.GetPointData().GetScalars()
    sc = numpy_support.vtk_to_numpy(scvtk)
    scmin, scmax = np.min(sc), np.max(sc)

    global plot2
    numobj=len(plot2.objects)
    if numobj>0:
        for i in range(numobj): plot2 -= plot2.objects[-1]
            
    print(len(plot2.objects))

    LVmesh = k3d.vtk_poly_data(sh2,
                             color_attribute=(scvtk.GetName(),scmin, scmax),
                             color_map=getattr(k3d.matplotlib_color_maps, cmsel.value),
                             color_range=[scmin, scmax])
    plot2 += LVmesh
    plot2 += RVmesh

    lvobj=plot2.objects[0]
    lvobj.color_map = getattr(k3d.matplotlib_color_maps, y)
    a = np.array([lvobj.color_range])
    pl.figure(figsize=(9, 0.5))
    img = pl.imshow(a, cmap=y)
    pl.gca().set_visible(False)
    cax = pl.axes([0.1, 0.2, 0.8, 0.6])
    pl.colorbar(orientation="horizontal", cax=cax)

"""        
@interact(x=cmsel)
def g(x):
    if len(plot2.objects)>0:
        lvobj=plot2.objects[0]
        lvobj.color_map = getattr(k3d.matplotlib_color_maps, x)
        a = np.array([lvobj.color_range])
        pl.figure(figsize=(9, 0.5))
        img = pl.imshow(a, cmap=x)
        pl.gca().set_visible(False)
        cax = pl.axes([0.1, 0.2, 0.8, 0.6])
        pl.colorbar(orientation="horizontal", cax=cax)
"""

Output()

interactive(children=(Checkbox(value=True, description='Display RV?'), Output()), _dom_classes=('widget-intera…

interactive(children=(Dropdown(description='Mesh map to display:', options=('WallThickness', 'SystolicExcursio…

'        \n@interact(x=cmsel)\ndef g(x):\n    if len(plot2.objects)>0:\n        lvobj=plot2.objects[0]\n        lvobj.color_map = getattr(k3d.matplotlib_color_maps, x)\n        a = np.array([lvobj.color_range])\n        pl.figure(figsize=(9, 0.5))\n        img = pl.imshow(a, cmap=x)\n        pl.gca().set_visible(False)\n        cax = pl.axes([0.1, 0.2, 0.8, 0.6])\n        pl.colorbar(orientation="horizontal", cax=cax)\n'

In [ ]:
plot2.objects[-1].color_map = getattr(k3d.paraview_color_maps, cms[0])

In [41]:
plot2 -= plot2.objects[0]

IndexError: list index out of range